## Introduction
This code realizes a process of tuning the parameters of a Multi-Layer Perceptron (MLP) neural network using Particle Swarm Optimization (PSO), and evaluates its performance on the task of predicting coal's Gross Calorific Value (GCV).

## Steps：
1.Import necessary libraries: pandas, sklearn, numpy, and pyswarm.

2.Load the original coal data from a CSV file.

3.Randomly sample 40% of the data.

4.Select features (‘Moisture’, ‘Volatile_matter’, ‘Std.Ash’) and the target variable (‘GCV’) from the sampled data.

5.Split the sampled data into training and testing sets, with a 20% testing size.

6.Standardize the sampled training data using StandardScaler.

7.Define an ANN model with a ReLU activation function and the Adam solver.

8.Use Particle Swarm Optimization (PSO) to optimize the number of neurons in the hidden layers of the ANN.

9.Train the ANN model with the best parameters found by PSO.

10.Predict the GCV for the test set using the optimized ANN model.

11.Calculate evaluation metrics (MAE, MSE, RMSE, R2, and MAPE) for the predictions on the sampled test data.

## Data Description
 The data being read from <coal_all.csv> contains 10 variables：
 
(Moisture, Volatile_matter, Fixed_Carbon, Ash, Hydrogen, Carbon, Nitrogen, Oxygen, Sulfur, GCV)

 Three sets of data (SET1, SET2, SET3) have been constructed based on different input variables.

 SET1 (Moisture, Volatile_matter, Ash, GCV)
 
 SET2 (Carbon, Oxygen, Sulfur, GCV)
 
 SET3 (Moisture, Volatile_matter, Ash, Hydrogen, Carbon, Oxygen, Sulfur, GCV)

In the following code, the SET1 sub-dataset is used. To utilize the other two datasets, simply specify the corresponding variable names in the statements.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import GridSearchCV

## Prepare training data
Due to the low performance of my computer, if all the original data are added to the optimization algorithm, it will take several days to complete. To improve the running efficiency, a scheme was adopted to randomly select a portion of data from the original data to optimize the parameters.

In [2]:
# Load the original data
data_original = pd.read_csv("F:/煤质论文2/coal_all.csv")

# Sample 40% of the data randomly
sampled_data_original = data_original.sample(frac=0.4, random_state=42)

# Select features and target from the sampled data
X_sampled = sampled_data_original[['Moisture', 'Volatile_matter', 'Std.Ash']]
y_sampled = sampled_data_original['GCV']

# Split the sampled data into training and testing sets
X_train_sampled, X_test_sampled, y_train_sampled, y_test_sampled = train_test_split(X_sampled, y_sampled, test_size=0.2, random_state=42)

# Standardize the sampled training data
scaler = StandardScaler()
X_train_sampled_scaled = scaler.fit_transform(X_train_sampled)
X_test_sampled_scaled = scaler.transform(X_test_sampled)
print(X_train_sampled_scaled.shape)

(363, 3)


## Use PSO for parameter optimization
mainly to optimize the number of hidden layers

In [ ]:
# PSO-ANN
from pyswarm import pso
def ann_obj_function(params):
   # Create an ANN model
    ann = MLPRegressor(hidden_layer_sizes=(int(params[0]), int(params[1])), activation='relu', solver='adam', max_iter=5000, random_state=42)
    ann.fit(X_train_sampled_scaled, y_train_sampled)
    y_pred = ann.predict(X_test_sampled)
    mse = mean_squared_error(y_test_sampled, y_pred)
    return mse
# Perform parameter optimization using Particle Swarm Optimization (PSO)
lb = [3, 3]  # Lower bound for the number of neurons in each hidden layer
ub = [24, 24]  # Upper bound for the number of neurons in each hidden layer
# Define the dimensions of the PSO as 2, because we have two parameters to optimize
xopt, fopt = pso(ann_obj_function, lb, ub)

# Create a model using the optimized parameters
optimized_neurons = (int(xopt[0]), int(xopt[1]))
ann_best = MLPRegressor(hidden_layer_sizes=optimized_neurons, 
                       activation='relu', solver='adam', max_iter=200, random_state=42)
ann_best.fit(X_train_sampled_scaled, y_train_sampled)

# Print the optimized parameters and results
optimized_params = ", ".join(f"{param}" for param in xopt)
print(f"Optimized parameters: {optimized_params}")
print(f"Best MSE: {fopt}")

## GCV prediction is made by using the determined parameters

In [ ]:
ann_best.fit(X_train_sampled_scaled,  y_train_sampled)
pso_ann_pred = ann_best.predict(X_test_scaled)

## Model Evaluation Metrics

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math
def calculate_mape(actual, predicted):
    return 100 * abs((actual - predicted).sum() / actual.sum())
# Calculate evaluation metrics using the sampled test data
pso_ann_mae_sampled = mean_absolute_error(y_test_sampled, pso_ann_pred_sampled)
pso_ann_mse_sampled = mean_squared_error(y_test_sampled, pso_ann_pred_sampled)
pso_ann_r2_sampled = r2_score(y_test_sampled, pso_ann_pred_sampled)
pso_ann_rmse_sampled = math.sqrt(pso_ann_mse_sampled)
pso_ann_mape_sampled = calculate_mape(y_test_sampled, pso_ann_pred_sampled)

pso_ann_mae_sampled, pso_ann_mse_sampled, pso_ann_rmse_sampled, pso_ann_r2_sampled, pso_ann_mape_sampled